#  프로젝트 - Movielens 영화 추천 실습

====================================================================================

# 목차
>### 1. 데이터 불러오기
>### 2. 데이터 전처리
>### 3. CSR matrix 만들기
>### 4. als_model 구성하기
>### 5. als_model 학습 및 결과 확인
>### 6. 영화 추천받기
>### 7. 회고
>### 8. Reference
>### 9. 자기다짐 및 아쉬운 점
---

## 루브릭 평가 문항

1. CSR matrix가 정상적으로 만들어졌다.
   - 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
   - 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
   - MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.
---

## 1. 데이터 불러오기

In [1]:
import numpy as np
import scipy
import implicit
import os
import pandas as pd
from implicit.als import AlternatingLeastSquares
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

- 필요한 모듈들을 불러오겠습니다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/exp/13_recommended/data/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/exp/13_recommended/data/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


- 앞으로 쓸 movie 데이터셋과 ratings 데이터셋을 불러왔습니다.

In [4]:
# 3점 이상만 남깁니다.
orig_size = len(ratings)
ratings = ratings[ratings['ratings']>=3]
filt_size = len(ratings)

print(f'원본 데이터 크기: {orig_size}, 정제 후 데이터 크기: {filt_size}')
print(f'정제된 데이터 비율 {filt_size / orig_size:.2%}')

원본 데이터 크기: 1000209, 정제 후 데이터 크기: 836478
정제된 데이터 비율 83.63%


- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다. 
- 이라는 LMS의 요구사항에 따라 3점 미만인 데이터들은 제거하였습니다.

In [5]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

- ratings라는 컬럼을 좀 더 보기 편하도록 counts라는 컬럼명으로 변경하겠습니다.

In [6]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


- timestamp 컬럼이 필요없어보이는군요 삭제해주도록 하겠습니다.

In [7]:
del ratings['timestamp']

---
## 2. 데이터 전처리

- LMS 요구사항
    - ratings에 있는 유니크한 영화 개수
    - ratings에 있는 유니크한 사용자 수
    - 가장 인기 있는 영화 30개(인기순)
    - 내가 선호하는 영화를 5가지 골라서 ratings에 추가해봅시다.

In [8]:
uni_user = ratings['user_id'].nunique()
uni_movie = ratings['movie_id'].nunique()

print(uni_user, uni_movie)

6039 3628


- user_id컬럼과 movie_id 의 nunique를 변수에 등록하였습니다.

In [9]:
movie_top30 = ratings.groupby('movie_id')['counts'].count().sort_values(ascending=False)[:30]
movie_top30

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: counts, dtype: int64

- movie_id를 counts순으로 정렬하여 탑 30을 정렬하였습니다.
- id인 정수형태로 되어있어 무슨 영화인지 알수가없군요? 한번 확인해보겠습니다.

In [10]:
for i in range(len(movie_top30)):
    if i == 0:
        ds = movie_top30.index[i]
        ds = movies.loc[movies['movie_id']==ds]
        df = pd.DataFrame(ds)
        df['counts'] = movie_top30.values[i]
    elif i != 0:
        ds = movie_top30.index[i]
        ds = movies.loc[movies['movie_id']==ds]
        ds['counts'] = movie_top30.values[i]        
        df = pd.concat([df,ds], axis=0)
        
df        

,movie_id,title,genre,counts
2789,2858,American Beauty (1999),Comedy|Drama,3211
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,2910
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,2885
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,2716
1959,2028,Saving Private Ryan (1998),Action|Drama|War,2561
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller,2509
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller,2498
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure,2473
1250,1270,Back to the Future (1985),Comedy|Sci-Fi,2460
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,2434


- 카운트 순으로 데이터프레임 top 30을 만들어봤습니다.
- 1위부터 아메리칸뷰티, 스타워즈 시리즈, 라이언 일병 구하기, 터미네이터, 양들의 침묵 등등 아주 쟁쟁한 영화들이 많군요.
- 잘 추출된것같습니다.

- 자 다음으로 제가 좋아하는 영화 5개를 ratings에 추가하라는군요!
- 일단 무슨 영화가 있는지 살펴볼까요?
- 저는 공포, 스릴러 영화를 좋아하니까 공포영화부터 살펴보겠습니다.

In [11]:
movies[movies['genre']=='Horror'][150:170]

,movie_id,title,genre
3582,3651,"Blood Spattered Bride, The (La Novia Ensangren...",Horror
3583,3652,City of the Living Dead (Paura nella città dei...,Horror
3640,3709,Sleepwalkers (1992),Horror
3663,3732,"Fury, The (1978)",Horror
3688,3757,Asylum (1972),Horror
3693,3762,Daughter of Dr. Jeckyll (1957),Horror
3703,3772,Hatchet For the Honeymoon (Rosso Segno Della F...,Horror
3762,3832,"Black Sabbath (Tre Volti Della Paura, I) (1963)",Horror
3767,3837,Phantasm II (1988),Horror
3768,3838,Phantasm III: Lord of the Dead (1994),Horror


- 재밌는 영화들이 많군요. 공포영화로 재밌게봤던 나이트메어와 헬레이저를 추가하였습니다.
- 그 다음으론 SF 영화도 좋아하는데요! 한번 살펴보겠습니다.

In [12]:
movies[movies['genre']=='Sci-Fi']

,movie_id,title,genre
673,680,Alphaville (1965),Sci-Fi
1181,1199,Brazil (1985),Sci-Fi
1188,1206,"Clockwork Orange, A (1971)",Sci-Fi
1281,1301,Forbidden Planet (1956),Sci-Fi
1530,1570,Tetsuo II: Body Hammer (1992),Sci-Fi
1941,2010,Metropolis (1926),Sci-Fi
1965,2034,"Black Hole, The (1979)",Sci-Fi
2457,2526,Meteor (1979),Sci-Fi
2509,2578,"Sticky Fingers of Time, The (1997)",Sci-Fi
2589,2658,"Flying Saucer, The (1950)",Sci-Fi


- 음.. 여긴 없군요 공상과학영화가 생각보다 많이 적군요?
- 한번 장르들을 살펴봐야겠습니다.

In [13]:
movies['genre'].unique()

array(["Animation|Children's|Comedy", "Adventure|Children's|Fantasy",
       'Comedy|Romance', 'Comedy|Drama', 'Comedy',
       'Action|Crime|Thriller', "Adventure|Children's", 'Action',
       'Action|Adventure|Thriller', 'Comedy|Drama|Romance',
       'Comedy|Horror', "Animation|Children's", 'Drama',
       'Action|Adventure|Romance', 'Drama|Thriller', 'Drama|Romance',
       'Thriller', 'Action|Comedy|Drama', 'Crime|Drama|Thriller',
       'Drama|Sci-Fi', 'Romance', 'Adventure|Sci-Fi', 'Adventure|Romance',
       "Children's|Comedy|Drama", 'Documentary', 'Drama|War',
       'Action|Crime|Drama', 'Action|Adventure', 'Crime|Thriller',
       "Animation|Children's|Musical|Romance", 'Action|Drama|Thriller',
       "Children's|Comedy", 'Drama|Mystery', 'Sci-Fi|Thriller',
       'Action|Comedy|Crime|Horror|Thriller', 'Drama|Musical',
       'Crime|Drama|Romance', 'Adventure|Drama', 'Action|Thriller',
       "Adventure|Children's|Comedy|Musical", 'Action|Drama|War',
       'Action|Adventur

- 장르가 상당히 많군요.. 눈에 띄는 장르는 'Action|Adventure|Sci-Fi', 'Horror|Sci-Fi', 'Action|Crime|Sci-Fi', 'Action|Horror|Sci-Fi|Thriller' 군요 한번 살펴보겠습니다.

In [14]:
movies[movies['genre']=='Action|Adventure|Sci-Fi']

,movie_id,title,genre
171,173,Judge Dredd (1995),Action|Adventure|Sci-Fi
313,316,Stargate (1994),Action|Adventure|Sci-Fi
325,329,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1335,1356,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi
1350,1371,Star Trek: The Motion Picture (1979),Action|Adventure|Sci-Fi
1351,1372,Star Trek VI: The Undiscovered Country (1991),Action|Adventure|Sci-Fi
1352,1373,Star Trek V: The Final Frontier (1989),Action|Adventure|Sci-Fi
1353,1374,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi
1354,1375,Star Trek III: The Search for Spock (1984),Action|Adventure|Sci-Fi


- 재밌는 영화들은 많으나 제 취향은 아니네요.. 넘어가겠습니다.

In [15]:
movies[movies['genre']=='Horror|Sci-Fi']

,movie_id,title,genre
194,196,Species (1995),Horror|Sci-Fi
328,332,Village of the Damned (1995),Horror|Sci-Fi
508,512,Robert A. Heinlein's The Puppet Masters (1994),Horror|Sci-Fi
956,968,Night of the Living Dead (1968),Horror|Sci-Fi
1314,1334,"Blob, The (1958)",Horror|Sci-Fi
1645,1692,Alien Escape (1995),Horror|Sci-Fi
1793,1862,Species II (1998),Horror|Sci-Fi
1855,1924,Plan 9 from Outer Space (1958),Horror|Sci-Fi
2187,2256,Parasite (1982),Horror|Sci-Fi
2308,2377,Lifeforce (1985),Horror|Sci-Fi


- 플라이가 포함되어있군요! 저것도 재밌게 봤던 영화였습니다.

In [16]:
movies[movies['genre']=='Horror|Thriller']

,movie_id,title,genre
308,311,Relative Fear (1994),Horror|Thriller
403,407,In the Mouth of Madness (1995),Horror|Thriller
732,742,Thinner (1996),Horror|Thriller
879,891,Halloween: The Curse of Michael Myers (1995),Horror|Thriller
1201,1219,Psycho (1960),Horror|Thriller
1312,1332,"Believers, The (1987)",Horror|Thriller
1334,1355,Nightwatch (1997),Horror|Thriller
1384,1407,Scream (1996),Horror|Thriller
1668,1717,Scream 2 (1997),Horror|Thriller
1822,1891,"Ugly, The (1997)",Horror|Thriller


- 몇개 없지만 명작인 로보캅이 있군요! 추가해주겠습니다.

In [17]:
movies[movies['genre']=='Action|Horror|Sci-Fi|Thriller']

,movie_id,title,genre
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller
1300,1320,Alien³ (1992),Action|Horror|Sci-Fi|Thriller
2219,2288,"Thing, The (1982)",Action|Horror|Sci-Fi|Thriller


- 여기도 명작인 에일리언이 있군요. 추가하겠습니다.

In [18]:
my_movie_name = ['Nightmare on Elm Street, A(1984)', 'Hellraiser(1987)','Fly, The(1986)', 'Robocop(1987)', 'Alien(1979)' ]
my_movies = [1347, 3917, 2455, 2985, 1214]

- 5가지 영화를 모두 선정했습니다. 아주 재밌는 영화들이 많군요!
- 일단 보기 편하도록 무비 이름 리스트와 무비 id 리스트를 따로 만들어 주었습니다.

In [19]:
ratings.tail()

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [20]:
ratings.user_id.max()

6040

- 마지막 user_id 가 6040이군요! 저는 6041로 추가해주도록 하겠습니다.

In [21]:
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_movies, 'counts': [5,5,4,5,5]})

In [22]:
ratings = ratings.append(my_playlist)

- 데이터 프레임 형식으로 만들어주고 각 영화에 별점을 매긴 후 ratings에 추가해주었습니다. 

In [23]:
ratings.tail(10)

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,1347,5
1,6041,3917,5
2,6041,2455,4
3,6041,2985,5
4,6041,1214,5


- 적용이 잘 된것을 확인할 수 있습니다.
- 제가 매긴 user_id, movie_id, counts까지 정상적으로 잘 들어갔군요
- 인덱스가 살짝 신경쓰이니 리셋해주도록 하겠습니다.

In [24]:
ratings = ratings.reset_index()
del ratings['index']

ratings.tail(10)

,user_id,movie_id,counts
836473,6040,1090,3
836474,6040,1094,5
836475,6040,562,5
836476,6040,1096,4
836477,6040,1097,4
836478,6041,1347,5
836479,6041,3917,5
836480,6041,2455,4
836481,6041,2985,5
836482,6041,1214,5


- 인덱스 정렬도 잘 되었고 깔끔하게 잘 정리되었군요!
- 이제 CSR matrix를 만들어보겠습니다.

---
## 3. CSR matrix 만들기

In [25]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movies))
csr_data

ValueError: row index exceeds matrix dimensions

- 어라?! 오류가 뜹니다
- 왜 그럴까요? 이유를 알아보았습니다.

In [26]:
print('영화 id 총 갯수 :', movies.movie_id.nunique())
print('ratings의 영화 id 갯수: ', ratings.movie_id.nunique())

영화 id 총 갯수 : 3883
ratings의 영화 id 갯수:  3628


In [27]:
print('ratings의 user_id 총 갯수', ratings.user_id.nunique())
print('ratings의 user_id 최대값', ratings.user_id.max())

ratings의 user_id 총 갯수 6040
ratings의 user_id 최대값 6041


- 이유를 찾았습니다
- 제가 생각하기로는 서로 맞지 않는 데이터 갯수로 인한 행렬의 형태 오류인듯 합니다.
- 그렇다면 총갯수, 최대 갯수에서 현재 갯수를 뺀 후 2를 곱해준다면 가능할듯 합니다.
- 왜냐하면 행렬은 x, y 로 구성되어 있으니까요.. 때문에 2를 곱해줍니다.

In [28]:
nunique_user = ratings.user_id.nunique()
max_user = ratings.user_id.max()
nunique_movie = movies.movie_id.nunique()
ratings_movie = ratings.movie_id.nunique()

In [29]:
m = (max_user - nunique_user) * 2
n = (nunique_movie - ratings_movie) * 2

In [30]:
print('user_id 차이: ', m)
print('movie_id 차이: ', n)

user_id 차이:  2
movie_id 차이:  510


- 잘 구해진듯 합니다. 이것을 m, n으로 취급하여 행렬 shape에 적용한다면 될듯합니다

In [31]:
csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user+m, num_movies+n))
csr_data

<6042x4138 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

- csr_matrix가 잘 생성되었습니다! 행렬의 형태 차이가 맞았군요!

---
## 4. als_model 구성하기

- AlternatingLeastSquares 를 사용하여 모델을 만들건데요! 일단 그 파라미터로 무엇이 있는지 확인해 보겠습니다.
  
  

-  AlternatingLeastSquares 클래스의 init 파라미터를 살펴보겠습니다.
- 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- 3. use_gpu : GPU를 사용할 것인지
- 4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
- 1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [32]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

csr_data_transpose = csr_data.T
csr_data_transpose

<4138x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

- als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해주었습니다.
- 이제 학습을 시켜보도록 하겠습니다.

---
## 5. als_model 학습 및 결과 확인

In [33]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

- 훈련이 끝났습니다. 학습이 잘 됬는지 확인해보도록 하겠습니다.

In [34]:
print(my_movie_name)
print(my_movies)

['Nightmare on Elm Street, A(1984)', 'Hellraiser(1987)', 'Fly, The(1986)', 'Robocop(1987)', 'Alien(1979)']
[1347, 3917, 2455, 2985, 1214]


- 벡터를 확인할 제 영화 목록을 가져왔습니다.

In [35]:
als_model.user_factors[6041]

array([ 0.04175149, -0.34165183, -0.5356145 , -0.47200683, -0.24756289,
       -0.8398471 , -0.30640134, -0.00137538,  0.29856238,  0.6956394 ,
       -0.00674405,  0.32488188,  0.5301642 , -0.5105078 , -0.53010106,
       -0.23722124, -0.16612154,  0.6977515 ,  0.20384867, -0.09365441,
        0.1450401 ,  0.557243  , -0.02830924,  0.1647612 ,  0.25642502,
        0.26277378,  0.6950124 , -0.13164322, -0.25570187,  0.03374013,
        0.24138594,  0.35508204,  0.14495842,  0.27827072,  0.08501954,
       -0.8057507 , -0.21147145,  0.16252013, -0.2819982 ,  0.0538465 ,
       -0.19901557, -0.17954065, -0.07850319,  0.0407069 ,  0.9322156 ,
        0.28359443,  0.28614333,  0.06597687, -0.6975243 ,  0.2535402 ,
        0.28199878,  0.39599162,  0.13104454, -0.03163331,  0.10273235,
       -0.41030553,  0.2786835 , -0.6603365 , -0.3893624 , -0.44290835,
       -0.00430288, -0.18870139, -0.19954531, -0.05551578,  0.31867   ,
        0.21938069,  0.6720175 ,  0.05083223,  0.26721302,  0.09

- 벡터가 잘 생성 되었군요!
- 이를 이용해 영화와의 관계를 확인해보겠습니다.

In [36]:
my_id = als_model.user_factors[6041]
my_movie_01 = als_model.item_factors[1347] 
my_movie_02 = als_model.item_factors[3917]
my_movie_03 = als_model.item_factors[2455]
my_movie_04 = als_model.item_factors[2985]
my_movie_05 = als_model.item_factors[1214]

- 제가 좋아하는 영화들을 모두 벡터화 하였습니다. 관계를 확인해 볼까요?

In [37]:
print('나와 나이트메어 영화: ', np.dot(my_id, my_movie_01))
print('나와 헬레이저 영화: ', np.dot(my_id, my_movie_02))
print('나와 플라이 영화: ', np.dot(my_id, my_movie_03))
print('나와 로보캅 영화: ', np.dot(my_id, my_movie_04))
print('나와 에일리언 영화: ', np.dot(my_id, my_movie_05))

나와 나이트메어 영화:  0.27852067
나와 헬레이저 영화:  0.21017338
나와 플라이 영화:  0.5172448
나와 로보캅 영화:  0.38046652
나와 에일리언 영화:  0.4707441


- 음 무언가 수치로 알려주긴 하는데.. 정확히 무슨 의미인지 알수가 없습니다.
- similar_items 를 이용해 영화간의 관계도를 구해보겠습니다.

In [38]:
nightmare_similar_movie = als_model.similar_items(1347, N=10)
nightmare_similar_movie

[(1347, 1.0),
 (1974, 0.84353465),
 (2517, 0.7447744),
 (1982, 0.74237335),
 (2513, 0.7342572),
 (1130, 0.7332307),
 (2787, 0.7129246),
 (1350, 0.7037122),
 (2867, 0.69778794),
 (2097, 0.6950102)]

- 정확도가 꽤 일치하는 있는 영화들이 있군요.
- 하지만 영화의 아이디만 봐서는 저게 무슨 영화를 뜻하는지 모르겠습니다.
- 함수를 만들어 한번 확인해보겠습니다.

In [39]:
def find_to_movie(movie_id):
    movie_name = movies.loc[movies['movie_id']==movie_id].title
    return movie_name

In [40]:
def idx_to_movie(vector, name='similarity'):
    for i in range(len(vector)):
        if i == 0:
            recommend = movies.loc[movies['movie_id'] == vector[i][0]].title
            recommend = pd.DataFrame(recommend)
            recommend[name] = round((vector[i][1]*100)/1,2)
        elif i != 0:
            y = movies.loc[movies['movie_id'] == vector[i][0]].title
            y = pd.DataFrame(y)
            y[name] = round((vector[i][1]*100)/1, 2)
            recommend = pd.concat([recommend,y], axis=0)
    return recommend

- 단순히 movie_id 만 넣어주면 이름을 리턴해주는 함수를 만들었습니다.
  
  
- 또한 데이터프레임 형태로 유사도를 추가한 형태의 함수를 만들었습니다. 한번 확인해보겠습니다.

In [41]:
find_to_movie(1347)

1326    Nightmare on Elm Street, A (1984)
Name: title, dtype: object

In [42]:
idx_to_movie(nightmare_similar_movie)

,title,similarity
1326,"Nightmare on Elm Street, A (1984)",100.00
1905,Friday the 13th (1980),84.35
2448,Christine (1983),74.48
1913,Halloween (1978),74.24
2444,Pet Sematary (1989),73.43
1114,"Howling, The (1980)",73.32
2718,Cat's Eye (1985),71.29
1329,"Omen, The (1976)",70.37
2798,Fright Night (1985),69.78
2028,Something Wicked This Way Comes (1983),69.50


- 한결 보기 좋게 보여주고 있군요! 제가 좋아한 나이트메어와 가장 유사한 영화로는 13일의 금요일이 있습니다. 그리고 크리스틴, 하울링, 할로윈, 캣츠아이 등등이 있는데 제가 아는 영화는 별로 없군요.
  
  
- 다른 영화도 확인해 볼까요?

In [43]:
find_to_movie(2985)

2916    Robocop (1987)
Name: title, dtype: object

In [44]:
robocop_similar_movie = als_model.similar_items(2985, N=10)
robocop_similar_movie

[(2985, 0.9999999),
 (3527, 0.7223851),
 (1129, 0.59180456),
 (2641, 0.5632775),
 (2640, 0.5485188),
 (3698, 0.52047664),
 (1240, 0.49217466),
 (2527, 0.47617108),
 (3704, 0.47015172),
 (2105, 0.44666255)]

In [45]:
idx_to_movie(robocop_similar_movie)

,title,similarity
2916,Robocop (1987),100.00
3458,Predator (1987),72.24
1113,Escape from New York (1981),59.18
2572,Superman II (1980),56.33
2571,Superman (1978),54.85
3629,"Running Man, The (1987)",52.05
1220,"Terminator, The (1984)",49.22
2458,Westworld (1973),47.62
3635,Mad Max Beyond Thunderdome (1985),47.02
2036,Tron (1982),44.67


- 프레데터, 뉴욕 탈출, 슈퍼맨, 매드맥스 등등을 알려주고 있군요!
  
  
- 아는 영화도 꽤 있는게 생각보다 유사도 파악을 아주 잘해주고 있는 모습입니다!

---
## 6. 영화 추천받기

- 그렇다면 제가 좋아하는 영화들로 추천을 받아보면 어떨까요?
- recommend 함수를 이용하여 확인해 보겠습니다.

In [46]:
my_favorite = als_model.recommend(6041, csr_data, N=10, filter_already_liked_items=True)

In [47]:
my_favorite

[(1200, 0.38368458),
 (1240, 0.3465697),
 (3527, 0.34299034),
 (2288, 0.30623016),
 (1387, 0.28237206),
 (1321, 0.2783666),
 (1129, 0.27093804),
 (2160, 0.26747167),
 (1994, 0.25701675),
 (1127, 0.2509969)]

- 추천해주는게 꽤 있군요.
- 이 영화들이 뭘까요?

In [48]:
idx_to_movie(my_favorite, name='recommend')

,title,recommend
1182,Aliens (1986),38.37
1220,"Terminator, The (1984)",34.66
3458,Predator (1987),34.30
2219,"Thing, The (1982)",30.62
1366,Jaws (1975),28.24
1301,"American Werewolf in London, An (1981)",27.84
1113,Escape from New York (1981),27.09
2091,Rosemary's Baby (1968),26.75
1925,Poltergeist (1982),25.70
1111,"Abyss, The (1989)",25.10


- 제가 좋아하는 영화인 터미네이터, 프레데터, 어비스, 죠스들이 있군요!
- 추천도는 조금 아쉽지만 추천은 굉장히 잘해주네요! 아주 신기합니다.
- 제가 모르는 영화들도 찾아보니 대부분 장르를 바탕으로 추천해주는 듯 합니다.

- 이번엔 기여도도 한번 확인해 보겠습니다.
- .explain을 이용하면 기여도를 확인할 수 있습니다!

In [49]:
find_to_movie(1387)

1366    Jaws (1975)
Name: title, dtype: object

In [50]:
explain = als_model.explain(6041, csr_data, itemid=1387)

- 기여도는 제가 제일 좋아한다고 했던 죠스를 기준으로 해볼까요?
- 제 영화들이 얼마나 관여했는지 보겠습니다.

In [51]:
explain[1]

[(1214, 0.2257817090573362),
 (1347, 0.050431052710251445),
 (2455, 0.02260616518256142),
 (3917, 0.002252814545654828),
 (2985, -0.02111106897509282)]

- 역시 이것만 봐서는 잘 모르겠군요.
- 확인을 해보겠습니다.

In [52]:
idx_to_movie(explain[1], name = 'contribution')

,title,contribution
1196,Alien (1979),22.58
1326,"Nightmare on Elm Street, A (1984)",5.04
2386,"Fly, The (1986)",2.26
3847,Hellraiser (1987),0.23
2916,Robocop (1987),-2.11


- 에일리언과 장르가 비슷한가보군요. 가장 많은 기여도를 보여줍니다.
- 로보캅이 음수의 기여도를 보여주는데요.. 음 아무래도 장르가 다르기 때문인거같군요. 음수라는건 아마 기여를 전혀 하지못했다는 뜻으로 보입니다.

In [53]:
find_to_movie(3527)

3458    Predator (1987)
Name: title, dtype: object

- 위에서 추천해줬던 프레데터에 대한 기여도도 확인해 보겠습니다.

In [54]:
explain = als_model.explain(6041, csr_data, itemid=3527)

In [55]:
idx_to_movie(explain[1], name = 'contribution')

,title,contribution
2916,Robocop (1987),17.77
1196,Alien (1979),11.79
2386,"Fly, The (1986)",5.16
1326,"Nightmare on Elm Street, A (1984)",-0.09
3847,Hellraiser (1987),-0.62


- 로보캅이 가장 높은 기여를 했군요.
- 여기서도 나이트메어와 헬레이저는 음수의 기여도를 보여줍니다.
- 아무래도 제 편향된 영화취향이.. 장르를 다양하게 가지지 못하다보니 이런 경우가 발생하는것 같군요!

---
## 7. 회고

### 이번 프로젝트를 하면서 어려웠던 점
>- 솔직히 이번 프로젝트는 크게 어렵진 않았지만 했던 프로젝트들 중에서 수를 이용한 방법은 또 처음이라 크게 와닿지 않아서 처음엔 살짝 헤메게 되었습니다. 또한 CSR matrix를 만들 때 shape가 맞지않아 자꾸 오류가 떠서 그부분에서도 헤맸지만 행렬의 형태를 생각하고 해결해보니 잘 되었습니다.

### 이번 프로젝트에서 학습한 내용
>- 공부한거라면 조금 더 수학적인 부분에서 학습을 좀 하게 된것 같습니다. 솔직하게 이 뒤에 있는 트랜스포머 챗봇때문에 여기에 크게 기여하면 안될것같아 많이 공부하지 않고 넘어가게 되었습니다.

### 알아낸 점이나 모호한 점
>- 알아낸 점, 모호한 점 역시나 크게 없었고 행렬의 형태, 수, 계산법 정도에 있어서 조금은 익숙해진듯 합니다. 아직 유사도, 추천도, 기여도 등 정확하게 이해되지 않고 모호하긴하나 이는 차차 알아갈 예정입니다.

### 루브릭 평가지표를 맞추기 위해 노력했던 점
>- __1. 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.__ 에 관해서는 제가 올바르게 한지는 모르겠지만 행렬의 아이템 갯수를 토대로 shape에 더해서 만들었습니다. 사실 이부분은 데이터 인덱싱을 해주면 해결되는 것인것은 알고 있었으나 굳이 하고싶지 않았습니다. 왜냐하면 인덱싱이란것은 각 id 같은 고유 이름에 번호를 부여하는 식인데 이미 user_id, movie_id에 고유한 번호가 부여되어 있고 이를 굳이 다시 맞춰주고싶지 않았습니다. 굳이? 왜? 라는 생각이 들었고 이미 인덱싱화 되있다고 판단해서 인덱싱은 진행하지 않았습니다. 때문에 맞추려고 노력은 하였으나 올바른 방법인지는 정확히 판단하진 못하겠습니다.
>- __2. 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.__ 의 상위 루브릭인 __MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.__ 와 연관지어 생각한다면 추천은 제가 생각했던것보다 훨씬 잘 해주었고 저의 영화 취향에 맞도록 잘 해주었다고 생각합니다. 이를 근거로 사용자와 아이템 벡터 내적수치 또한 의미있다고 판단할 수 있다고 생각합니다.
>- __3. MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.__ 에 관하여서는 유저 선호도, 아이템간 유사도, 기여도 모두 측정해보았고 유의미한 성과가 있었다고 생각합니다. 제가 예상한것보다 훨씬 추천도 잘 해주었고 유사도 또한 판단은 잘 했으나 기여도 부분에서는.. 조금 의문이 들긴합니다. 제가 ratings에 추가한게 5개밖에 안되는 영화갯수때문이기도 한것같고 또한 이 기여도에 counts가 미치는 영향에 대해서는 이해하지 못했습니다.


### 루브릭 평가지표를 달성하지 못했다면 이유
>- 루브릭 평가지표를 달성하지 못했다면 이유로는 아마 1번 루브릭과 3번 루브릭에 있지 않을까 싶습니다. 올바르게 행렬의 shape가 형성되었는가? 이에 대해서는 제 생각은 맞다 이지만 정답과는 다를 수 있다고 판단합니다. 제 나름의 생각과 추론 과정을 통해 나온 저의 생각이었지만 이게 맞는지 여부에 대해서는 불분명하기 때문입니다.
>  
>  
>- 또한 3번 루브릭에 관하여서는 기여도에 대해 정확히 이해하지 못했고 의미를 정확하게 파악하지 못하여서 애매한 부분이 있어 그렇지 않을까 생각합니다.


## 8. Reference

- https://junstar92.tistory.com/265 csr matrix 블로그

## 9. 자기다짐 및 아쉬운 점

>- 이번 프로젝트는 아쉬운 점이 꽤 많았습니다. 집중해서 좀 더 좋은 성능을 이끌어내고 싶었으나 아직 수가 표현해주는 뜻을 잘 모르겠고 85만큼의 추천이라면 이게 85% 추천인지? 1을 기준으로 0.85 만큼이라는것인지 어떤 의미인지 정확한 느낌이 오지 않습니다.
>  
>  
>- 이제 유플러스 AI 경진대회에도 나가게되는데 그 내용이 추천시스템에 대한 머신러닝 또한 딥러닝인데 이 프로젝트를 열심히 해야했으나 생각보다 진행에 막히는 부분도 없었고 다음 프로젝트인 트랜스포머 챗봇에 너무 신경쓰여서 이번 프로젝트에 집중을 크게 못한부분이 많이 아쉽습니다.
>  
>  
>- 하지만, 해야하는 부분이니 프로젝트가 끝나더라도 이번 프로젝트는 집중하여 다시 한번 진행해 볼 생각입니다.